In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-27 16:11:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-27 16:11:52 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [66]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [11]:
vine_df

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [67]:
# Filter dataframe on total_vote greater than or equal to 20
totalvotes_df = vine_df.filter("total_votes>=20")
totalvotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|     

In [68]:
# Filter dataframe on the number of helpful_votes/total_votes is equal to or greater than 50%
helpful_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [69]:
# Filter the DataFrame where a review was written as part of the Vine program (paid)
paid_vineprogram_df = helpful_df.filter(helpful_df["vine"] == "Y")
paid_vineprogram_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [70]:
# Filter the DataFrame where a review was not written as part of the Vine program (unpaid)
unpaid_vineprogram_df = helpful_df.filter(helpful_df["vine"] == "N")
unpaid_vineprogram_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [19]:
from pyspark.sql.functions import count


In [71]:
# The total number of reviews
review_count = helpful_df.count()
print("Total Number of helpful reviews: %2f" % review_count)

vine_review_count = paid_vineprogram_df.count()
print("Total Number of helpful reviews part of the vine program: %f" % vine_review_count)

non_vine_review_count = unpaid_vineprogram_df.count()
print("Total Number of helpful reviews not part of the vine program: %f" % non_vine_review_count)


Total Number of helpful reviews: 1685.000000
Total Number of helpful reviews part of the vine program: 0.000000
Total Number of helpful reviews not part of the vine program: 1685.000000


In [72]:
# calculate the total number of 5-star reviews and 5 star reviews for vine and non vine programs

fivestar_rating_count = helpful_df.filter(helpful_df['star_rating']=='5').count()
print("The total number of helpful 5 star reviews: %2f"  %fivestar_rating_count)

vine_fivestar_rating = paid_vineprogram_df.filter(paid_vineprogram_df['star_rating']=='5').count()
print("The total number of helpful 5 star reviews part of the vine program: %2f" % vine_fivestar_rating)

nonvine_fivestar_rating = unpaid_vineprogram_df.filter(unpaid_vineprogram_df['star_rating']=='5').count()
print("The total number of helpful 5 star reviews not part of the vine program: %2f" % nonvine_fivestar_rating)




The total number of helpful 5 star reviews: 631.000000
The total number of helpful 5 star reviews part of the vine program: 0.000000
The total number of helpful 5 star reviews not part of the vine program: 631.000000


In [73]:
# calculate 5 star review percentages for two types of review

vine_fivestar_percent = (vine_fivestar_rating/fivestar_rating_count)*100
nonvine_fivestar_percent = (nonvine_fivestar_rating/fivestar_rating_count)*100


# **Review Summary for Digital Video Games**

In [74]:
print("Total number of digital video games reviews with votes greater tnan or equal to 20: %2f" %totalvotes_df.count())
print("Total number of helpful reviews (helpful votes/total votes): %2f" % review_count)
print("\n")

print("Total number of helpful reviews part of the vine program: %f" % vine_review_count)
print("Total number of helpful reviews not part of the vine program: %f" % non_vine_review_count)
print("\n")

print("Total number of helpful 5 star reviews: %2f"  %fivestar_rating_count)
print("Total number of helpful 5 star reviews part of the vine program: %2f" % vine_fivestar_rating)
print("Total number of helpful 5 star reviews NOT part of the vine program: %2f" % nonvine_fivestar_rating)
print("\n")

print("Percent of 5-Star reviews part of the vine program: %f" %vine_fivestar_percent)
print("Percent of 5-Star reviews NOT part of the vine program: %f" %nonvine_fivestar_percent)

Total number of digital video games reviews with votes greater tnan or equal to 20: 3342.000000
Total number of helpful reviews (helpful votes/total votes): 1685.000000


Total number of helpful reviews part of the vine program: 0.000000
Total number of helpful reviews not part of the vine program: 1685.000000


Total number of helpful 5 star reviews: 631.000000
Total number of helpful 5 star reviews part of the vine program: 0.000000
Total number of helpful 5 star reviews NOT part of the vine program: 631.000000


Percent of 5-Star reviews part of the vine program: 0.000000
Percent of 5-Star reviews NOT part of the vine program: 100.000000
